### Import

In [1]:
# Import from packages
import pandas as pd
import numpy as np
import math
import os
import matplotlib.pyplot as plt
from mplsoccer import Pitch
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

### config.py

In [34]:
# Directory with all files of dataset
directory = '/Users/gian-andrea/Documents/Masterarbeit (offline)/statsbomb360-euro2020/events/'

# Filepath to one file of dataset
filepath = directory + '3788745.json'

# List with columns relevant for shots dataframe
ls_shots = ['possession_team', 'shot', 'location']

# Dive radius of gk
dive_radius = 3

In [39]:
# Create dataframe
df_test = pd.read_json(filepath)

# Display dataframe
df_test

,id,index,period,timestamp,minute,second,type,possession,possession_team,play_pattern,...,duel,interception,off_camera,shot,goalkeeper,out,foul_committed,foul_won,block,substitution
0,68cc0b37-a3d6-4585-a3d0-873f6af5f9ff,1,1,2022-06-10 00:00:00.000,0,0,"{'id': 35, 'name': 'Starting XI'}",1,"{'id': 768, 'name': 'England'}","{'id': 1, 'name': 'Regular Play'}",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,7db89c9d-c666-47d4-bb16-db02f138ee5e,2,1,2022-06-10 00:00:00.000,0,0,"{'id': 35, 'name': 'Starting XI'}",1,"{'id': 768, 'name': 'England'}","{'id': 1, 'name': 'Regular Play'}",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4bd02a7e-9ab5-4c10-993b-48875aa54b73,3,1,2022-06-10 00:00:00.000,0,0,"{'id': 18, 'name': 'Half Start'}",1,"{'id': 768, 'name': 'England'}","{'id': 1, 'name': 'Regular Play'}",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,22da729b-6b2f-4fdc-9e91-587af4be8807,4,1,2022-06-10 00:00:00.000,0,0,"{'id': 18, 'name': 'Half Start'}",1,"{'id': 768, 'name': 'England'}","{'id': 1, 'name': 'Regular Play'}",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,de1529bc-c1f4-43fa-abb9-a802789f823b,5,1,2022-06-10 00:00:00.259,0,0,"{'id': 30, 'name': 'Pass'}",2,"{'id': 768, 'name': 'England'}","{'id': 9, 'name': 'From Kick Off'}",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3361,8e5f4119-3763-4b22-ab07-2e6f7f149120,3362,2,2022-06-10 00:49:31.551,94,31,"{'id': 2, 'name': 'Ball Recovery'}",149,"{'id': 785, 'name': 'Croatia'}","{'id': 4, 'name': 'From Throw In'}",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3362,63f73643-d21d-42d1-b8f4-25b2e43aae7b,3363,2,2022-06-10 00:49:31.718,94,31,"{'id': 16, 'name': 'Shot'}",149,"{'id': 785, 'name': 'Croatia'}","{'id': 4, 'name': 'From Throw In'}",...,NaN,NaN,NaN,"{'statsbomb_xg': 0.041979983000000005, 'end_lo...",NaN,NaN,NaN,NaN,NaN,NaN
3363,869aa987-913e-4153-a4d6-fdc6eaa1f250,3364,2,2022-06-10 00:49:32.438,94,32,"{'id': 23, 'name': 'Goal Keeper'}",149,"{'id': 785, 'name': 'Croatia'}","{'id': 4, 'name': 'From Throw In'}",...,NaN,NaN,NaN,NaN,"{'end_location': [0.9, 39.5], 'type': {'id': 3...",NaN,NaN,NaN,NaN,NaN
3364,6a9dc901-f95f-4091-b1d4-7142a58ec27b,3365,2,2022-06-10 00:49:35.780,94,35,"{'id': 34, 'name': 'Half End'}",149,"{'id': 785, 'name': 'Croatia'}","{'id': 4, 'name': 'From Throw In'}",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
# Initialize variables
filecount = 0
eventcount = 0

# Iterate over all files in directory
for filename in os.listdir(directory):

    # Save current filepath
    filepath = os.path.join(directory, filename)
    # Check if current file has json ending
    if filename.split('.')[1] == 'json':

        # Create dataframe from current json file
        df_file = pd.read_json(filepath)
        
        # Update counters
        eventcount = eventcount + len(df_file)
        filecount = filecount + 1

# Calculate average
avg_events = eventcount / filecount

# Display result
avg_events

3778.156862745098

### utils.py

In [4]:
# Create dataframe including all json files
def create_dataframe(directory):
    
    # Create empty lists
    ls_possession_team = []
    ls_type = []
    ls_shot = []
    ls_location = []

    # Iterate over all files in directory
    for filename in os.listdir(directory):

        # Save current filepath
        filepath = os.path.join(directory, filename)
        # Check if current file has json ending
        if filename.split('.')[1] == 'json':

            # Create dataframe from current json file
            df_file = pd.read_json(filepath)
    
            # Add columns of current dataframe as lists to existing lists
            ls_possession_team.extend(df_file['possession_team'].tolist())
            ls_type.extend(df_file['type'].tolist())
            ls_shot.extend(df_file['shot'].tolist())
            ls_location.extend(df_file['location'].tolist())

    # Create empty dataframe and add completed lists to dataframe
    df = pd.DataFrame()
    df['possession_team'] = ls_possession_team
    df['type'] = ls_type
    df['shot'] = ls_shot
    df['location'] = ls_location

    # Return dataframe
    return df

# Apply method and display dataframe
df_all = create_dataframe(directory)
print('Number of rows:', len(df_all))
df_all.head(1)

Number of rows: 192686


,possession_team,type,shot,location
0,"{'id': 776, 'name': 'Denmark'}","{'id': 35, 'name': 'Starting XI'}",NaN,NaN


In [5]:
df_all

,possession_team,type,shot,location
0,"{'id': 776, 'name': 'Denmark'}","{'id': 35, 'name': 'Starting XI'}",NaN,NaN
1,"{'id': 776, 'name': 'Denmark'}","{'id': 35, 'name': 'Starting XI'}",NaN,NaN
2,"{'id': 776, 'name': 'Denmark'}","{'id': 18, 'name': 'Half Start'}",NaN,NaN
3,"{'id': 776, 'name': 'Denmark'}","{'id': 18, 'name': 'Half Start'}",NaN,NaN
4,"{'id': 776, 'name': 'Denmark'}","{'id': 30, 'name': 'Pass'}",NaN,"[60.0, 40.0]"
...,...,...,...,...
192681,"{'id': 773, 'name': 'Switzerland'}","{'id': 23, 'name': 'Goal Keeper'}",NaN,"[1.0, 40.0]"
192682,"{'id': 772, 'name': 'Spain'}","{'id': 16, 'name': 'Shot'}","{'one_on_one': True, 'statsbomb_xg': 0.76, 'en...","[107.8, 40.0]"
192683,"{'id': 772, 'name': 'Spain'}","{'id': 23, 'name': 'Goal Keeper'}",NaN,"[1.0, 40.0]"
192684,"{'id': 772, 'name': 'Spain'}","{'id': 34, 'name': 'Half End'}",NaN,NaN


In [ ]:
# Create dataframe filtered for one specific type of event
def filter_event(df, ls):

    # Create event dataframe
    df = df[ls].dropna().reset_index(drop = True)
    
    # Unzip location to coordinates
    df['x_loc'], df['y_loc'] = zip(*df['location'])

    # Return event dataframe
    return df

# Apply method and display dataframe
df_filter_shots = filter_event(df_all, ls_shots)
print('Number of rows:', len(df_filter_shots))
df_filter_shots.head(1)

In [ ]:
# Create shots dataframe with additional data
def complete_shots(df):
    
    # Create shots dataframe
    df = filter_event(df, ls_shots)

    # Create empty lists
    ls_possession_team = []
    ls_shot = []
    ls_location = []
    ls_x_loc = []
    ls_y_loc = []
    ls_outcome = []
    ls_x_gk = []
    ls_y_gk = []
    ls_players = []

    # Iterate over all rows in dataframe
    for j in range (0, len(df)):
        
        # Check if current row has freeze_frame information
        if 'freeze_frame' in df.iloc[j]['shot']:
            
            # Create and reset nested players list
            ls_players_nested = []
            
            # Add values to lists
            ls_possession_team.append(df.iloc[j]['possession_team'])
            ls_shot.append(df.iloc[j]['shot'])
            ls_location.append(df.iloc[j]['location'])
            ls_x_loc.append(df.iloc[j]['x_loc'])
            ls_y_loc.append(df.iloc[j]['y_loc'])
            
            # Add nested outcome values to list
            ls_outcome.append(df.iloc[j]['shot']['outcome']['name'])
            
            # Save shot information from current freeze_frame
            shot = df.iloc[j]['shot']['freeze_frame']
            # Iterate over all players in one freeze_frame
            for i in range (0, len(shot)):
                # Add gk coordinates to lists
                if shot[i]['position']['name'] == 'Goalkeeper' and shot[i]['teammate'] == False:
                    ls_x_gk.append(shot[i]['location'][0])
                    ls_y_gk.append(shot[i]['location'][1])
                # Add other player locations to lists
                else:
                    ls_players_nested.append(shot[i]['location'])
            # Add list or empty string to list
            if len(ls_players_nested) > 0:
                ls_players.append(ls_players_nested)
            else:
                ls_players.append('')
    
    # Add lists to dataframe
    df = pd.DataFrame()
    df['possession_team'] = ls_possession_team
    df['shot'] = ls_shot
    df['outcome'] = ls_outcome
    df['location'] = ls_location
    df['x_loc'] = ls_x_loc
    df['y_loc'] = ls_y_loc
    df['x_gk'] = ls_x_gk
    df['y_gk'] = ls_y_gk
    df['players'] = ls_players
    
    # Return complete shots dataframe
    return df

# Apply method and display dataframe
df_complete_shots = complete_shots(df_all)
print('Number of rows:', len(df_complete_shots))
df_complete_shots.head(1)

In [ ]:
# Create shots dataframe with additional data about players in shot polygon
def polygon_players(df):
    
    # Create complete shots dataframe and exclude shots without other player locations
    df = complete_shots(df).where(df_complete_shots['players'] != '').dropna().reset_index(drop = True)
    
    # Create empty list
    ls_players = []
        
    # Iterate over all rows in dataframe
    for j in range(0, len(df)):
        # Create and reset nested players list
        ls_players_nested = []
        # Iterate over all players in one shot
        for i in range (0, len(df.iloc[j]['players'])):
            # Add points inside polygon to list
            point = Point(df.iloc[j]['players'][i][0], df.iloc[j]['players'][i][1])
            polygon = Polygon([(df.iloc[j]['x_loc'], df.iloc[j]['y_loc']), (120, 36), (120, 44)])
            if polygon.contains(point) == True:
                ls_players_nested.append(df.iloc[j]['players'][i])
        
        # Add list or empty string to list
        if len(ls_players_nested) > 0:
            ls_players.append(ls_players_nested)
        else:
            ls_players.append('')
    
    # Add list to dataframe
    df['polygon_players'] = ls_players
    
    # Return dataframe with all players in polygon
    return df

# Apply method and display dataframe
df_polygon_players = polygon_players(df_all)
print('Number of rows:', len(df_polygon_players))
df_polygon_players.head(2)

In [ ]:
"""
# Calculate angle between the two lines from event location to goalposts
def angle(x, y):
    
    # Calculate slope from event location to goalposts
    m1 = (36-y)/(120-x)
    m2 = (44-y)/(120-x)
    
    # Return absolute value of angle in radians
    return abs((math.atan((m1-m2)/(1+m1*m2))))

# Apply method and display value
print('Angle:', math.degrees(angle(119, 40.9)))
"""

In [ ]:
# Calculate angle between the two lines from event location to goalposts
def angle(x, y):
    
    # Calculate slope from event location to goalposts
    m1 = (36-y)/(120-x)
    m2 = (44-y)/(120-x)
    
    # Return absolute value of angle in radians
    return abs(math.atan(m1) - math.atan(m2))

# Apply method and display value
print('Angle:', math.degrees(angle(119, 40.9)))

In [ ]:
# Calculate slope of bisector
def bisector_slope(x, y):
    
    # x, y = origin = event location
    # x_p, y_p = point = lower goalpost location
    # x_r, y_r = rotated point
    
    # Define variables
    ang = angle(x, y)/2
    x_p = 120
    y_p = 36
    
    # Rotate point counterclockwise by given angle in radians around given origin
    x_r = x + math.cos(ang) * (x_p - x) - math.sin(ang) * (y_p - y)
    y_r = y + math.sin(ang) * (x_p - x) + math.cos(ang) * (y_p - y)
    
    # Return slope from event location to rotated point
    return (y_r-y)/(x_r-x)

# Apply method and display value
print('Bisector slope:', bisector_slope(112, 40))

In [ ]:
# Create dataframe with togkp
def get_togkp(df):
    
    # Create polygon players dataframe
    df = polygon_players(df)

    # Create empty list
    ls_x_opt = []
    ls_y_opt = []

    # Iterate over all rows in dataframe
    for i in range(0, len(df)):

        # Define linear equation of bisector
        x_loc = df.iloc[i]['x_loc']
        y_loc = df.iloc[i]['y_loc']
        m_loc = bisector_slope(x_loc, y_loc)
        q_loc = y_loc - m_loc*x_loc
        
        # Bisector angle in radians
        ang = angle(x_loc, y_loc)/2

        # If central shot
        if m_loc == 0:
            x_shift = (math.cos(ang)*dive_radius)/math.sin(ang)
            x_opt = x_loc + x_shift
            y_opt = 40
        # If no central shot
        else:
            # Define linear equation perpendicular to bisector
            d1 = math.dist([x_loc, y_loc], [120, 36])
            d2 = math.dist([x_loc, y_loc], [120, 44])
            if d1 < d2:
                y_p = 36
            else:
                y_p = 44
            x_p = 120
            m_p = -(1/m_loc)
            q_p = y_p - m_p*x_p

            # Calculate intersection of two linear equations
            y_opt = (q_loc - q_p*m_loc/m_p)/(1-(m_loc/m_p))
            x_opt = (y_opt-q_p)/m_p

        # If dive radius is not covered
        if math.dist([x_p, y_p], [x_opt, y_opt]) > dive_radius:

            # Distance from event location to optimal location to cover dive radius
            distance = (math.cos(ang)*dive_radius)/math.sin(ang)

            # Shift from shot location to optimal location
            x_shift = distance/(math.sqrt(1+m_loc*m_loc))
            y_shift = x_shift*m_loc

            # Optimal location
            x_opt = x_loc + x_shift
            y_opt = y_loc + y_shift
        
        # Add value to list
        ls_x_opt.append(x_opt)
        ls_y_opt.append(y_opt)

    # Add list to dataframe
    df['x_opt'] = ls_x_opt
    df['y_opt'] = ls_y_opt

    # Return togkp dataframe
    return df

# Apply method and display dataframe
df_togkp = get_togkp(df_all)
print('Number of rows:', len(df_togkp))
df_togkp.head(2)

In [ ]:
# Create dataframe with all impossible togkp locations
df_false = df_togkp.where(df_togkp['x_opt'] < df_togkp['x_loc']).dropna().reset_index(drop = True)

# Display df_fals with old index
df_togkp.where(df_togkp['x_opt'] < df_togkp['x_loc']).dropna()

In [ ]:
# Create dataframe with all impossible togkp locations
df_false = df_togkp.where(df_togkp['x_opt'] > 120).dropna().reset_index(drop = True)

# Display df_fals with old index
df_togkp.where(df_togkp['x_opt'] > 120).dropna()

In [ ]:
# Calculate distance between gkp and togkp
def get_distance(df):
    
    # Create togkp dataframe
    df = get_togkp(df)
    
    # Create empty list
    ls_distance = []
    
    # Iterate over all rows
    for i in range(0, len(df)):
        # Save coordinates
        x1 = df.at[i, 'x_opt']
        y1 = df.at[i, 'y_opt']
        x2 = df.at[i, 'x_gk']
        y2 = df.at[i, 'y_gk']
        # Add value to list
        ls_distance.append(math.dist([x1, y1], [x2, y2]))
    
    # Add list to dataframe
    df['distance'] = ls_distance

    # Return distance dataframe
    return df
    
# Apply method and display dataframe
df_distance = get_distance(df_all)
print('Number of rows:', len(df_distance))
df_distance.head(2)

In [ ]:
# Plot one shot
def plot_shot(df, row):
    
    # Plot soccer pitch
    pitch = Pitch(pitch_color='grass', line_color='white', stripe=True)
    fig, ax = pitch.draw()

    # Player location
    x_loc = df.at[row, 'x_loc']
    y_loc = df.at[row, 'y_loc']
    plt.scatter(x_loc, y_loc, color = 'blue')

    # Goalkeeper location
    x_gk = df.at[row, 'x_gk']
    y_gk = df.at[row, 'y_gk']
    plt.scatter(x_gk, y_gk, color = 'red')
    
    # Other player location
    for i in range(0, len(df.iloc[row]['polygon_players'])):
        x_p = df.iloc[row]['polygon_players'][i][0]
        y_p = df.iloc[row]['polygon_players'][i][1]
        plt.scatter(x_p, y_p, color = 'yellow')

    # TOGKP
    x_opt = df.at[row, 'x_opt']
    y_opt = df.at[row, 'y_opt']
    plt.scatter(x_opt, y_opt, color = 'orange')

    # Line between player location and gp1
    point1 = df.at[row, 'location']
    point2 = [120, 36]

    x_values = [point1[0], point2[0]]
    y_values = [point1[1], point2[1]]

    plt.plot(x_values, y_values, color = 'black')

    # Line between player location and gp2
    point1 = df.at[row, 'location']
    point2 = [120, 44]

    x_values = [point1[0], point2[0]]
    y_values = [point1[1], point2[1]]

    plt.plot(x_values, y_values, color = 'black')
    
    """
    # Plot bisector
    point1 = df.at[row, 'location']
    point2 = [x_opt, y_opt]

    x_values = [point1[0], point2[0]]
    y_values = [point1[1], point2[1]]

    plt.plot(x_values, y_values, color = 'black')
    """
    
    # Scale plot
    plt.axis('scaled')
    plt.xlim([-5, 125])
    plt.ylim([-5, 85])

    # Display plot
    plt.show()
    
    # Display player and gk location
    print('x_loc:', x_loc, 'y_loc:', y_loc)
    print('x_gk:', x_gk, 'y_gk:', y_gk)
    print('x_opt:', x_opt, 'y_opt:', y_opt)

In [ ]:
def plot_all(df):
    
    # Plot soccer pitch
    pitch = Pitch(pitch_color='grass', line_color='white', stripe=True)
    fig, ax = pitch.draw()
    
    # Plot all shots
    for i in range(0, len(df)):
        # Player location in blue
        x_loc = df.at[i, 'x_loc']
        y_loc = df.at[i, 'y_loc']
        plt.scatter(x_loc, y_loc, color = 'blue')
        
    # Scale plot
    plt.axis('scaled')
    plt.xlim([-5, 125])
    plt.ylim([-5, 85])

    # Display plot
    plt.show()

In [ ]:
"""
# Completed shot dataframe to relevant shots dataframe 
def relevant_shots(df):
    
    # Create complete shot dataframe
    df = complete_shots(df)
    
    # Return relevant shots dataframe
    return df.where((df['outcome'] == 'Goal') | (df['outcome'] == 'Saved') | (df['outcome'] == 'Saved To Post')).dropna().reset_index(drop = True)

# Apply method and display dataframe
df_relevant_shots = relevant_shots(df_all)
df_relevant_shots
"""

### data.py

In [ ]:
# Apply method and display dataframe
df_final = get_distance(df_all)
print('Number of rows:', len(df_final))
df_final.head(2)

In [ ]:
# Apply method and display dataframe
df_1v1 = df_final.where(df_final['polygon_players'] == '').dropna()
print('Number of rows:', len(df_1v1))
df_1v1.head(1)

### plot.py

In [ ]:
# Plot one shot
plot_shot(df_final, 141)

In [ ]:
# Plot all shots
plot_all(df_false)

### Testing

In [ ]:
def check_deviation(df):
    
    df_saved = df.where(df['outcome'] == 'Saved').dropna().reset_index(drop = True)
    print('Distance for saves:', df_saved['distance'].mean())
    
    df_nogoal = df.where(df['outcome'] != 'Goal').dropna().reset_index(drop = True)
    print('Distance for nogoals:', df_nogoal['distance'].mean())

    df_goal = df.where(df['outcome'] == 'Goal').dropna().reset_index(drop = True)
    print('Distance for goals:', df_goal['distance'].mean())

# Apply method
check_deviation(df_1v1)

In [ ]:
"""
# Check which dive_radius value gets smallest distance between gkp and togkp

# Initilaize variables
mean_distance = 100
count = 0
dive_radius_copy = dive_radius
dive_radius = 3.3

while count < 3:
    
    # Create dataframe with current dive radius
    df_distance = get_distance(df_all)
    
    # If mean distance is smaller update parameters and reset count to 0
    if df_distance['distance'].mean() < mean_distance:
        mean_distance = df_distance['distance'].mean()
        result = dive_radius
        count = 0
    # If mean distance is larger add 1 to count
    else:
        count = count + 1
    
    # Increase dive radius
    dive_radius = dive_radius + 0.1

# Recreate original dive radius
dive_radius = dive_radius_copy

# Print results
print("Final dive radius:", result)
print("Final mean distance:", mean_distance)
"""